# WFSS光谱模拟以进行污染校正

这个笔记本演示了基本技术，以模拟WFSS（宽场分光谱）分散光谱，给定来自成像模式曝光的源位置和来自附带WFSS曝光的广义世界坐标系统（gWCS）。

这样的模拟在应用于场中的所有源时，对于估计重叠光谱的污染以及在尝试估计背景时遮罩分散光谱轨迹至关重要。

这个笔记本基于更简单的<a href="https://github.com/spacetelescope/jdat_notebooks/blob/main/notebooks/NIRCam/NIRCam_WFSS_Box_extraction/BoxExtraction_using_Grismconf_CRDS.ipynb">盒提取笔记本</a>，在其中我们介绍了光谱提取的一般概念以及<A HREF="https://github.com/npirzkal/GRISMCONF">GRISMCONF</A>模块，该模块为WFSS模式的gWCS模型提供了低级接口。

模拟方法相对简单，首先从成像模式曝光开始。

* 我们首先确定哪些像素包含源信号以及这些像素中的信号水平。

* 然后，我们使用成像模式文件中的 gWCS 和 WFSS 观测数据来计算每个像素在 WFSS 观测框架中的对应位置。

* 接下来，我们使用 GRISMCONF 函数和波长向量来模拟源的色散。对于每个源像素，我们遍历波长向量并计算每个波长值色散到的位置。我们根据每个输入像素的通量以 $F_{\lambda}$ 单位（$erg/s/cm^2/A$）计算与每个波长位置相关的信号水平。

* 由于计算出的波长值坐标通常不会与探测器像素网格对齐，我们使用 <a href="https://github.com/spacetelescope/pypolyclip">Sutherland-Hodgman 算法</A> 来计算每个投影色散像素与每个探测器像素重叠的部分。然后将每个信号相加。

作者：N. Pirzkal <br>

创建日期：2024年12月12日

## 目录

1. [设置 CRDS 路径和服务器](#Set-CRDS-Path-and-Server)

2. [包导入](#Package-Imports)

3. [定义函数和参数](#Define-Functions-and-Parameters)

4. [下载数据](#Download-Data)

5. [运行管道步骤](#Run-Pipeline-Steps)

6. [检测源](#Detect-Sources)

7. [模拟单个源的光谱](#Simulate-spectrum-of-one-source)

      * [在成像和 WFSS 数据中定位源](#Locate-source-in-imaging-and-WFSS-data)

      * [获取波长信息](#Get-wavelength-information)

      * [模拟单个像素的色散](#Simulate-the-dispersion-of-a-single-pixel)

      * [对我们源的所有像素进行色散处理](#Disperse-all-the-pixels-for-our-source)

## 设置 CRDS 路径和服务器

在运行管道步骤之前，我们需要确保我们的CRDS环境已正确配置。这包括定义一个CRDS缓存目录，用于存放校准管道将使用的参考文件。

如果本地CRDS缓存的根目录尚未设置，它将会在主目录中创建。在导入crds包或任何依赖于crds的包之前，必须完成此步骤。

In [ ]:
import os  # 导入os模块，用于与操作系统交互

In [ ]:
# 检查本地 CRDS 缓存目录是否已设置。

# 如果没有设置，则将其设置为用户主目录
if (os.getenv('CRDS_PATH') is None):
    os.environ['CRDS_PATH'] = os.path.join(os.path.expanduser('~'), 'crds')  # 设置 CRDS_PATH 环境变量

# 检查 CRDS 服务器 URL 是否已设置。如果没有设置，则进行设置。
if (os.getenv('CRDS_SERVER_URL') is None):
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'  # 设置 CRDS_SERVER_URL 环境变量

# 输出当前使用的 CRDS 路径和上下文
print('CRDS local filepath:', os.environ['CRDS_PATH'])  # 打印本地 CRDS 文件路径
print('CRDS file server:', os.environ['CRDS_SERVER_URL'])  # 打印 CRDS 文件服务器 URL

# 在设置好所需的环境变量后导入 crds
from crds import client  # 导入 crds 客户端
if client.get_crds_server() != os.environ['CRDS_SERVER_URL']:  # 检查当前 CRDS 服务器是否与环境变量一致
    client.set_crds_server('https://jwst-crds.stsci.edu')  # 如果不一致，则设置为指定的 CRDS 服务器

## 包导入

In [ ]:
from astropy.convolution import convolve  # 导入天文数据卷积函数

from astropy.io import fits  # 导入FITS文件读写模块

from copy import deepcopy  # 导入深拷贝函数

import matplotlib.pyplot as plt  # 导入绘图库

import numpy as np  # 导入NumPy库

import requests  # 导入HTTP请求库

from scipy.sparse import coo_matrix  # 导入稀疏矩阵的COO格式

import tqdm  # 导入进度条库

import grismconf  # 导入光栅配置模块

from jwst import datamodels  # 导入JWST数据模型模块

from jwst.assign_wcs import AssignWcsStep  # 导入WCS分配步骤

from jwst.flatfield import FlatFieldStep  # 导入平场校正步骤

from jwst.photom import PhotomStep  # 导入光度测量步骤

from photutils.background import Background2D, MedianBackground  # 导入背景处理模块

from photutils.segmentation import make_2dgaussian_kernel  # 导入生成二维高斯核的函数

from photutils.segmentation import detect_sources  # 导入源检测函数

from pypolyclip import clip_multi  # 导入多边形裁剪函数

## 定义函数和参数

定义一个函数，通过MAST API下载指定文件到当前目录。该函数包含身份验证逻辑，但由于此示例使用的是公共数据，因此不需要MAST API令牌。

In [ ]:
def get_jwst_file(name, mast_api_token=None, overwrite=False):
    """Retrieve a JWST data file from MAST archive.
    
    Parameters
    ----------
    name : str
        Name of the file to download from MAST
        
    mast_api_token : str
        MAST API token. Required only for proprietary data
        
    overwrite : bool
        If True and the requested file already exists locally, the file will not be downloaded. IF False,
        the file will be downloaded
    """

    # 如果文件已经存在于本地，则不重新下载，除非用户设置了overwrite关键字
    if os.path.isfile(name):
        if not overwrite:
            print(f'{name} already exists locally. Skipping download.')  # 文件已存在，跳过下载
            return
        else:
            print(f'{name} exists locally. Re-downloading.')  # 文件已存在，重新下载

    mast_url = "https://mast.stsci.edu/api/v0.1/Download/file"  # MAST下载文件的API URL
    params = dict(uri=f"mast:JWST/product/{name}")  # 设置请求参数，指定要下载的文件

    if mast_api_token:  # 如果提供了API令牌
        headers = dict(Authorization=f"token {mast_api_token}")  # 设置请求头，包含API令牌
    else:
        headers = {}  # 如果没有提供API令牌，设置为空字典

    r = requests.get(mast_url, params=params, headers=headers, stream=True)  # 发送GET请求以下载文件
    r.raise_for_status()  # 检查请求是否成功，如果不成功则抛出异常

    with open(name, "wb") as fobj:  # 以二进制写入模式打开文件
        for chunk in r.iter_content(chunk_size=1024000):  # 分块读取内容
            fobj.write(chunk)  # 将读取的内容写入文件

    if os.path.isfile(name):  # 检查文件是否成功下载
        print(f"{name} successfully downloaded")  # 打印成功下载的消息

定义一个函数，该函数将对输入的速率文件运行 `assign_wcs` 和平场校正。

In [ ]:
def run_pipeline_steps(filename):

    """Run the assign_wcs, flat field, and photom calibration steps on the given file.

    If the file contains WFSS data, trick the pipeline to use the imaging mode flat

    field reference file.

    

    Parameters

    ----------

    filename : str

        Name of the input file upon which the steps will be run

        

    Returns

    -------

    filename : str

        Name of the output file saved by the pipeline steps

        

    photom : jwst.datamodels.ImageModel

        Datamodel instance containing the calibrated data

    """

    # 调用AssignWcsStep对输入文件进行WCS分配
    assign_wcs = AssignWcsStep.call(filename)

    # 为了将成像模式的平场参考文件应用于数据，
    # 我们需要通过暂时将光阑值更改为CLEAR来欺骗CRDS
    reset_pupil = False

    # 检查光阑是否为GRISM，如果是，则进行处理
    if 'GRISM' in assign_wcs.meta.instrument.pupil:
        true_pupil = deepcopy(assign_wcs.meta.instrument.pupil)  # 备份原始光阑值
        assign_wcs.meta.instrument.pupil = 'CLEAR'  # 将光阑值设置为CLEAR
        reset_pupil = True  # 标记需要重置光阑值

    # 运行平场步骤
    flat = FlatFieldStep.call(assign_wcs, save_results=True)

    # 运行光度校准步骤，以填充WFSS灵敏度的名称
    photom = PhotomStep.call(flat, save_results=True)

    # 在平场处理完成后，将光阑值设置回原始值
    if reset_pupil:
        photom.meta.instrument.pupil = true_pupil  # 恢复原始光阑值
        photom.save(photom.meta.filename)  # 保存校准后的数据

    # 返回输出文件的名称以及数据模型
    return photom.meta.filename, photom

In [ ]:
def show_2d_spec(data, xlim=(200, 240), ylim=(1705, 1730), vmin=0, vmax=3):
    """显示带有颜色条的2D图像。旨在显示2D真实和模拟光谱

    参数
    ----------
    data : numpy.ndimage
        2D图像
    xlim : tup
        显示的开始和结束x坐标的2元组
    ylim : tup
        显示的开始和结束y坐标的2元组
    vmin : float
        对应于最小显示比例的信号
    vmax : float
        对应于最大显示比例的信号
    """

    fig, ax = plt.subplots(1, 1, figsize=(15, 3))  # 创建一个15x3英寸的图形和一个子图

    cax = ax.imshow(data, origin="lower", aspect='auto', cmap='viridis', vmin=vmin, vmax=vmax)  # 显示2D图像

    ax.set_xlim(xlim[0], xlim[1])  # 设置x轴范围，(0, 700)可以查看整个光谱

    ax.set_ylim(ylim[0], ylim[1])  # 设置y轴范围

    plt.xticks(range(xlim[0], xlim[1], 5))  # 设置x轴刻度

    plt.xlabel("Dispersion coordinate (pixel)")  # 设置x轴标签

    plt.ylabel("Cross dispersion coordinate (pixel)")  # 设置y轴标签

    colorbar = fig.colorbar(cax, orientation='vertical', pad=0.01)  # 添加颜色条

    colorbar.ax.set_ylabel('Signal', labelpad=10, rotation=270)  # 设置颜色条的标签

## 下载数据

我们从一对简单的成像（imaging）和宽场光谱（wfss）文件开始。这些文件是手动选择的，指向天空中的同一领域，并使用相同的NIRCam模块、通道和交叉滤光器。

In [ ]:
# 首先，从MAST下载成像和WFSS文件

imaging_file = "jw01076109001_02102_00001_nrcalong_cal.fits"  # 成像文件的名称

wfss_file = "jw01076109001_02101_00001_nrcalong_rate.fits"  # WFSS文件的名称

get_jwst_file(imaging_file)  # 下载成像文件

get_jwst_file(wfss_file)  # 下载WFSS文件

## 运行管道步骤

我们想要为WFSS数据分配一个WCS（世界坐标系统），应用平场（flat-field），并进行通量校准（flux calibrate）。为此，我们使用上面定义的`run_pipeline_steps()`函数。这将调用适当的处理步骤。由于平场与波长无关，我们将成像模式的平场应用于WFSS文件。

In [ ]:
# 运行 AssignWcsStep、FlatFieldStep 和 PhotomStep 在 WFSS 速率文件上

wfss_file, wfss_data = run_pipeline_steps(wfss_file)  # 调用管道步骤处理 WFSS 文件

# 从数据模型实例中提取 WFSS 像素数据

wfss_data = wfss_data.data  # 获取 WFSS 数据中的像素信息

In [ ]:
# 打印处理后的WFSS文件名
print(f"Pipeline-processed WFSS file is {wfss_file}")

从成像数据和宽场光谱扫描（WFSS）数据中读取一些信息。我们需要知道我们正在查看的模块（module）、通道（channel）、交叉滤光片（cross filter）和光栅（grism）。此外，我们还需要找到将成像校准文件的表面亮度单位转换为 $erg/s/cm^2/A$ 所需的值。

In [ ]:
img_hdr0 = fits.getheader(imaging_file, 0)  # 获取成像文件的第一个头部信息

img_hdr1 = fits.getheader(imaging_file, 1)  # 获取成像文件的第二个头部信息

wfss_hdr = fits.getheader(wfss_file)  # 获取WFSS文件的头部信息

In [ ]:
FILTER = img_hdr0["FILTER"]  # 从图像头信息中获取滤光片信息

MODULE = img_hdr0["MODULE"]  # 从图像头信息中获取模块信息

PUPIL = img_hdr0["PUPIL"]  # 从图像头信息中获取光阑信息

PIXAR_SR = img_hdr1["PIXAR_SR"]  # 从图像头信息中获取像素面积（以球面弧度为单位）

# 打印出滤光片、模块、光阑和像素面积的信息
print(f"IMAGING FILTER: {FILTER}, MODULE: {MODULE}, PUPIL: {PUPIL}, Sise of pixel: {PIXAR_SR} steradians")

In [ ]:
FILTER = wfss_hdr["FILTER"]  # 从wfss_hdr字典中获取FILTER值

MODULE = wfss_hdr["MODULE"]  # 从wfss_hdr字典中获取MODULE值

PUPIL = wfss_hdr["PUPIL"]    # 从wfss_hdr字典中获取PUPIL值

print(f"WFSS FILTER: {FILTER}, MODULE: {MODULE}, PUPIL: {PUPIL}")  # 打印WFSS的FILTER、MODULE和PUPIL信息

请注意，在这个简单的例子中，成像数据和宽场光谱扫描（WFSS）数据使用相同的交叉滤光片，并且当然使用的是相同的NIRCam模块。

计算成像数据中像素值（单位为MJy/sr）与$F_{\lambda}$单位（单位为$erg/s/cm^2/A$，每个像素）的转换。将我们校准的成像文件中的值乘以这个值（称为PHOTFLAM）即可确定在我们的成像数据中检测到的每个像素的$F_{\lambda}$值。

NIRCam滤光片的中心波长列在<a href="https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-instrumentation/nircam-filters#gsc.tab=0">NIRCam滤光片</a>的表2和表3中。

In [ ]:
Pivot_wavelength = 44010  # 波长，单位为埃（Angstroms）

PHOTFLAM = 1e6 * PIXAR_SR / 3.3356e4 / Pivot_wavelength**2  # 计算PHOTFLAM值

print(f'PHOTFLAM is {PHOTFLAM}')  # 输出PHOTFLAM的值

## 检测源

由于我们希望对构成给定源的每个像素进行分散，首先需要获取图像中所有对象的分割图（segmentation map）。我们遵循 photutils 中的 <a href="https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.detect_sources.html">detect_sources() 函数文档</a> 中介绍的一般方法来创建分割图。

In [ ]:
# 从FITS文件中读取成像数据
imaging_data = fits.getdata(imaging_file)

In [ ]:
# 创建一个二维背景模型，并进行减法处理

bkg_estimator = MedianBackground()  # 使用中位数背景估计器

# 创建一个二维背景，指定图像数据、块大小和滤波器大小
bkg = Background2D(imaging_data, (50, 50), filter_size=(21, 31), bkg_estimator=bkg_estimator)

# 从原始图像数据中减去背景
imaging_data -= bkg.background  # 更新图像数据，减去背景

In [ ]:
# 使用背景的均方根（RMS）来设置源检测的阈值

threshold = 50 * bkg.background_rms  # 将背景的均方根乘以50以计算阈值

In [ ]:
# 使用2D高斯核对图像进行卷积

# 创建一个标准差为3.0，大小为5的2D高斯核
kernel = make_2dgaussian_kernel(3.0, size=5)

# 将图像数据与高斯核进行卷积
convolved_data = convolve(imaging_data, kernel)

In [ ]:
# 在成像数据中检测源

# 调用 detect_sources 函数，传入卷积后的数据、阈值和最小像素数，返回源的分段图
segment_map = detect_sources(convolved_data, threshold, npixels=10)

下面我们可以看到，分割图显示了在探测器上分布着大量的源。

In [ ]:
# 显示分割图
plt.imshow(segment_map, origin="lower", vmin=1, vmax=2)  # 使用imshow函数显示分割图，设置原点在下方，vmin和vmax用于设置颜色映射的范围

## 模拟一个源的光谱

在这里，我们展示了如何模拟仅一个源的色散。为了模拟完整的WFSS（宽场光谱扫描）观测，我们在此展示的过程需要对场中的每个源进行一次。模拟所有色散光谱也是在后续提取过程中估计色散背景水平时遮蔽光谱的一种方法，同时它还允许估算重叠光谱造成的光谱污染量。

在这个例子中，我们选择坐标为 (x,y) = (405,1465) 的源，并展示如何模拟其光谱。

我们首先获取其分割图 ID，并创建一个与成像数据中该源相关的所有像素的列表。

In [ ]:
xd, yd = 405, 1465  # 定义坐标xd和yd，分别为405和1465

ID = segment_map.data[yd, xd]  # 从segment_map数据中获取指定坐标的对象ID

print(f"Object ID is: {ID}")  # 打印对象ID

### 在成像和宽场光谱扫描（WFSS）数据中定位源

使用成像模式文件，创建一个源的所有像素坐标列表，以及它们的通量值（单位为 MJy/SR）。

In [ ]:
ok = segment_map.data == ID  # 创建一个布尔数组，判断segment_map中每个元素是否等于ID

yds, xds = np.nonzero(ok)  # 获取ok数组中为True的元素的行和列索引

cds = imaging_data[ok]  # 从imaging_data中提取出对应ok为True的元素

显示来自成像数据的源及其分割图。在左侧，我们可以看到该源似乎是一个点源。分割图显示大约6x6像素的集合，这些像素被识别为该源的一部分。

In [ ]:
min_x = np.min(xds)  # 计算xds中的最小值，赋值给min_x

max_x = np.max(xds)  # 计算xds中的最大值，赋值给max_x

min_y = np.min(yds)  # 计算yds中的最小值，赋值给min_y

max_y = np.max(yds)  # 计算yds中的最大值，赋值给max_y

fig, axs = plt.subplots(1, 2, figsize=(15, 5))  # 创建一个1行2列的子图，设置图像大小为15x5英寸

axs[0].imshow(imaging_data[min_y:max_y + 1, min_x:max_x + 1], origin="lower")  # 显示源图像数据，y轴从下方开始

axs[1].imshow(segment_map.data[min_y:max_y + 1, min_x:max_x + 1], origin="lower")  # 显示分割图像数据，y轴从下方开始

axs[0].set_title("Source")  # 设置第一个子图的标题为“Source”

axs[1].set_title("Segmentation map")  # 设置第二个子图的标题为“Segmentation map”

我们对这个源的所有信息都在成像数据的参考框架内。但我们需要知道这个源在WFSS（宽场光谱扫描）观测中的通量位置。这是通过成像和WFSS观测的gWCS（广义天文坐标系统）来处理的。通过这些，我们将成像数据中每个源像素的位置转换为WFSS数据中相应的像素位置。

In [ ]:
# 使用JWST数据模型打开成像文件，并获取WCS信息
imaging_wcs = datamodels.open(imaging_file)  # 打开成像文件并创建数据模型对象

imaging_to_world = imaging_wcs.meta.wcs.get_transform('detector', 'world')  # 获取从探测器坐标到世界坐标的转换

In [ ]:
# 获取与WFSS文件相关的WCS信息

wfss_wcs = datamodels.open(wfss_file)  # 打开WFSS文件并加载其数据模型

wfss_to_pix = wfss_wcs.meta.wcs.get_transform('world', 'detector')  # 获取从世界坐标到探测器坐标的转换

作为参考，下面我们看到成像数据中的像素坐标。对应于源的像素范围在 x 值为 402 到 408 之间，y 值为 1461 到 1468 之间。

In [ ]:
plt.scatter(xds, yds)  # 绘制散点图，x轴为xds，y轴为yds

plt.xlabel("Imaging columns")  # 设置x轴标签为“成像列”

plt.ylabel("Imaging rows")  # 设置y轴标签为“成像行”

计算每个源的成像模式输入像素的赤经（R.A.）和赤纬（Dec）

In [ ]:
# 将图像坐标转换为世界坐标
ras, decs = imaging_to_world(xds, yds)  # xds和yds是图像中的坐标，ras和decs是对应的世界坐标

现在计算在WFSS数据中**未分散**源的像素坐标。在这种情况下，gWCS需要输入波长和光谱阶。这些相同的值由转换函数返回。在这里，我们选择波长为3.56微米，光谱阶为1。在这种情况下返回的像素坐标不依赖于波长或光谱阶，因此可以使用任何值。

In [ ]:
# 将天文坐标转换为像素坐标
xs, ys, xxx, yyy = wfss_to_pix(ras, decs, 3.56, 1)  # 调用函数wfss_to_pix，将ras和decs转换为像素坐标，3.56和1为参数

显示WFSS数据中**未色散**源的像素位置。请注意，该位置与成像数据中的位置有显著不同。在这种情况下，源位于x值为117到123之间，y值为1740和1746之间。

In [ ]:
import matplotlib.pyplot as plt  # 导入绘图库

# 绘制散点图，x轴为xs，y轴为ys
plt.scatter(xs, ys)  # 绘制散点图

# 设置x轴标签为"WFSS columns"
plt.xlabel("WFSS columns")  # 设置x轴标签

# 设置y轴标签为"WFSS rows"
plt.ylabel("WFSS rows")  # 设置y轴标签

# 显示图形
plt.show()  # 显示绘制的图形

### 获取波长信息

在模拟这个分散光谱时，我们需要考虑被分散光的波长。因此，上述每个像素将数值上分散到一系列离散波长上。

为了获取所需的波长信息，我们初始化一个 grismconf Config 对象。这个对象包含了描述分散器分散特性的相关信息和多项式，以及相应的逆灵敏度曲线。

In [ ]:
C = grismconf.Config(wfss_file)  # 创建一个grismconf.Config对象，传入wfss_file作为参数

显示逆灵敏度，包括波长范围和灵敏度的形状。其单位为 DN/s 每 $F_{\lambda}$ （$erg/s/cm^2/A$）。曲线在大约 3.85 微米到 5.05 微米之间显示出显著的灵敏度。

In [ ]:
plt.plot(C.SENS_data["+1"][0], C.SENS_data["+1"][1])  # 绘制数据，x轴为波长，y轴为DN/s per erg/s/cm^2/A

plt.grid()  # 添加网格线

plt.xlabel("Wavelength (micron)")  # 设置x轴标签为“波长（微米）”

plt.ylabel(r"DN/s per $erg/s/cm^2/A$")  # 设置y轴标签为“每$erg/s/cm^2/A$的DN/s”

我们使用grimconf配置对象快速获取与色散器对应的波长范围。该信息位于WRANGE属性中。

In [ ]:
wmin = C.WRANGE["+1"][0]  # 获取波长范围的最小值

wmax = C.WRANGE["+1"][1]  # 获取波长范围的最大值

print(f"The wavelength range to consider is {wmin} to {wmax} microns")  # 打印波长范围

计算每个像素的波长色散（dispersion）。Grismconf可以为我们提供关于色散的导数，单位为波长和像素，均是相对于$t$参数的。关于$t$参数的详细信息，请参见<a href="https://github.com/spacetelescope/jdat_notebooks/blob/main/notebooks/NIRCam/NIRCam_WFSS_Box_extraction/BoxExtraction_using_Grismconf_CRDS.ipynb">Box Extraction Notebook</a>。如文中所述，$t$是一个归一化参数，其中$t = 0$和$t = 1$分别对应于分散光谱的蓝边和红边。

虽然色散在光谱覆盖的探测器区域内略有变化，但我们使用$t$值为0.5时的色散，这对应于光谱范围的中间位置。

In [ ]:
# 计算光谱的色散（dlam），使用DDISPL和DDISPX函数
dlam = C.DDISPL("+1", 1000, 1000, 0.5) / C.DDISPX("+1", 1000, 1000, 0.5)

# 打印色散值，单位为埃（Angstroms），并将其转换为每个像素的值
print(f"Dispersion is {dlam * 10000} Angstroms per pixel")

接下来，创建一个波长值数组，以便计算模拟光谱。为此，我们必须选择一个波长步长。理想情况下，这个步长应该小于光栅的固有色散。因此，我们选择上述计算的色散值的一半。

In [ ]:
dlam = dlam / 2  # 将波长间隔减半

lams = np.arange(wmin, wmax, dlam)  # 生成从wmin到wmax的波长数组，步长为dlam

print(f"We are using {len(lams)} values of wavelength")  # 打印使用的波长值的数量

In [ ]:
# 打印波长数组的第一个和最后一个元素
print(f'First and last elements of the wavelength array: {"%.5f" % lams[0]} microns, {"%.5f" % lams[-1]} microns')

### 模拟单个像素的色散

有了波长信息，我们可以为每个对象像素创建一个模拟的色散。

下面，我们展示了单个像素的处理过程。我们选择一个相对靠近源中心的像素。

In [ ]:
i = 22  # 设置像素索引为22

# 打印出用于单像素色散的像素坐标 (x, y)
print(f"Use pixel (x, y) = ({xs[i]}, {ys[i]}) for single pixel dispersion")

我们首先计算与我们考虑的波长（lams）对应的 $t$ 值。有关 $t$ 值的更多背景信息，请参阅 Box Extraction 笔记本。

In [ ]:
ts = C.INVDISPL("+1", xs[i], ys[i], lams)  # 调用C库中的INVDISPL函数，计算给定坐标和波长的逆位移

接下来，我们创建一个多边形数组，表示来自我们选择的输入像素的分散信号的位置。

下面的单元格计算在WFSS观测中，我们像素的左下角坐标数组。

In [ ]:
# 计算新的 x 坐标，使用 DISPX 函数并加上原始 x 坐标
xgsA = C.DISPX("+1", xs[i], ys[i], ts) + xs[i]

# 计算新的 y 坐标，使用 DISPY 函数并加上原始 y 坐标
ygsA = C.DISPY("+1", xs[i], ys[i], ts) + ys[i]

以下三个单元格计算其他三个角的位置：

In [ ]:
# 计算xgsB的值，使用DISPX函数并加上当前xs[i]的值加1
xgsB = C.DISPX("+1", xs[i] + 1, ys[i], ts) + xs[i] + 1

# 计算ygsB的值，使用DISPY函数并加上当前ys[i]的值
ygsB = C.DISPY("+1", xs[i] + 1, ys[i], ts) + ys[i]

In [ ]:
# 计算 x 方向的位移
xgsC = C.DISPX("+1", xs[i] + 1, ys[i] + 1, ts) + xs[i] + 1  # 调用 DISPX 函数计算 x 方向的位移并加上当前 x 坐标

# 计算 y 方向的位移
ygsC = C.DISPY("+1", xs[i] + 1, ys[i] + 1, ts) + ys[i] + 1  # 调用 DISPY 函数计算 y 方向的位移并加上当前 y 坐标

In [ ]:
# 计算 x 方向的位移
xgsD = C.DISPX("+1", xs[i], ys[i] + 1, ts) + xs[i]  # 调用 DISPX 函数，传入参数并加上当前 x 坐标

# 计算 y 方向的位移
ygsD = C.DISPY("+1", xs[i], ys[i] + 1, ts) + ys[i] + 1  # 调用 DISPY 函数，传入参数并加上当前 y 坐标和 1

稍微重新组织一下内容，以包含多边形的角点列表，这些角点由 pypolyclip 模块使用，以计算它们与 WFSS 观测的像素坐标的重叠。由于我们正在查看单个输入源像素，因此我们在多个不同的波长值下进行计算，因此结果是一个包含多个像素/多边形的列表，这些像素/多边形将投影到我们的 WFSS 矩形像素网格上。

In [ ]:
# 创建一个二维列表pxs，每个子列表包含xgsA, xgsB, xgsC, xgsD中对应索引ii的元素
pxs = [[xgsA[ii], xgsB[ii], xgsC[ii], xgsD[ii]] for ii in range(len(xgsA))]

In [ ]:
# 使用列表推导式创建一个二维列表pys
pys = [[ygsA[ii], ygsB[ii], ygsC[ii], ygsD[ii]] for ii in range(len(ygsA))]  # 遍历ygsA的索引，构建包含四个元素的子列表

下面我们创建一个图形，展示分散像素的结果位置。我们使用一个波长数组对单个输入像素进行分散，该波长数组相对于光栅的原生色散超采样了2倍。该波长数组被转换为一个像素位置数组，覆盖在WFSS探测器网格上。

为了清晰起见，我们在色散方向上放大了一个宽40像素的区域。这显示了我们的单个成像模式像素将在该区域沿着几乎水平的线（以彩虹色方框表示）进行分散。通过在下面的单元中更改xlim值，缩小该图将显示与分散输入像素对应的整个像素集。为了保持一致性，我们在接下来的两个部分中为其他图形展示了类似的宽40像素的图。

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 3))  # 创建一个15x3英寸的子图

for i in tqdm.tqdm(range(len(pxs))):  # 遍历pxs列表的每个元素

    tx = pxs[i]  # 获取当前的pxs元素

    tx.append(pxs[i][0])  # 将当前元素的第一个值添加到tx的末尾，以闭合曲线

    ty = pys[i]  # 获取当前的pys元素

    ty.append(pys[i][0])  # 将当前元素的第一个值添加到ty的末尾，以闭合曲线

    plt.plot(tx, ty)  # 绘制tx和ty的曲线

plt.xticks(range(0, len(pxs)))  # 设置x轴刻度为从0到pxs长度的范围

plt.xlim(200, 240)  # 设置x轴的显示范围为200到240（可以改为(0, 700)以查看整个光谱）

plt.xlabel("WFSS columns")  # 设置x轴标签为"WFSS columns"

plt.ylabel("WFSS Rows")  # 设置y轴标签为"WFSS Rows"

plt.grid()  # 显示网格

plt.xlabel("Dispersion coordinate (pixel)")  # 设置x轴标签为"Dispersion coordinate (pixel)"

plt.ylabel("Cross ispersion coordinate (pixel)")  # 设置y轴标签为"Cross ispersion coordinate (pixel)"

我们现在可以使用 `pypolyclip.clip_multi` 来计算每个分散像素（上面的彩色框）落在 WFSS 图像像素网格（上面的灰色网格）上的多少部分。有关详细信息，请访问 <A HREF="https://github.com/spacetelescope/pypolyclip">pypolyclip 页面</A>。

In [ ]:
# 调用clip_multi函数，传入pxs和pys数组，以及图像尺寸[2048, 2048]
xc, yc, area, slices = clip_multi(pxs, pys, [2048, 2048])  # xc: x坐标, yc: y坐标, area: 区域, slices: 切片

在接下来的部分，我们将对所有像素重复这一工作流程，并利用区域信息来缩放所有输出像素的通量值。

### 将我们源的所有像素分散开来

请注意，上面的图显示了一个单一源像素被色散的情况。对于一个完整的源，每个输入源像素应以类似的方式被色散，从而导致多个色散像素对WFSS图像中每个探测器像素的最终计数产生贡献。我们还需要计算并将每个输入像素的适当通量（以DN/s为单位）归属到每个WFSS探测器像素。以下是我们为所选对象进行的计算。

我们必须跟踪所有信息，例如波长和落在WFSS模拟像素阵列上的原始成像像素通量的比例，以便为每个成像像素提供准确的数据。

In [ ]:
xcs = []  # 存储x坐标
ycs = []  # 存储y坐标
alams = []  # 存储波长
flams = []  # 存储光通量

all_pxs = []  # 存储所有像素的x坐标
all_pys = []  # 存储所有像素的y坐标
all_flams = []  # 存储所有光通量
all_counts = []  # 存储所有计数

# 在WFSS参考框架中循环遍历输入源像素
for i in tqdm.tqdm(range(len(xs))):

    # 使用每个像素中的成像光通量计算输入的DN/s和flam单位
    counts = cds[i]  # 获取当前像素的计数
    flam = counts * PHOTFLAM  # 计算光通量

    # 使用len(lams)波长来分散此像素。这将导致len(lams)个投影
    # 像素贡献给最终的WFSS数据
    ts = C.INVDISPL("+1", xs[i], ys[i], lams)  # 计算反向位移
    xgsA = C.DISPX("+1", xs[i], ys[i], ts) + xs[i]  # 计算分散后的x坐标A
    ygsA = C.DISPY("+1", xs[i], ys[i], ts) + ys[i]  # 计算分散后的y坐标A
    xgsB = C.DISPX("+1", xs[i] + 1, ys[i], ts) + xs[i] + 1  # 计算分散后的x坐标B
    ygsB = C.DISPY("+1", xs[i] + 1, ys[i], ts) + ys[i]  # 计算分散后的y坐标B
    xgsC = C.DISPX("+1", xs[i] + 1, ys[i] + 1, ts) + xs[i] + 1  # 计算分散后的x坐标C
    ygsC = C.DISPY("+1", xs[i] + 1, ys[i] + 1, ts) + ys[i] + 1  # 计算分散后的y坐标C
    xgsD = C.DISPX("+1", xs[i], ys[i]+1, ts) + xs[i]  # 计算分散后的x坐标D
    ygsD = C.DISPY("+1", xs[i], ys[i]+1, ts) + ys[i] + 1  # 计算分散后的y坐标D

    # 使用分散像素的角落，计算它们重叠的WFSS像素，以及重叠的面积
    pxs = [[xgsA[ii], xgsB[ii], xgsC[ii], xgsD[ii]] for ii in range(len(xgsA))]  # 存储x坐标的四个角
    pys = [[ygsA[ii], ygsB[ii], ygsC[ii], ygsD[ii]] for ii in range(len(ygsA))]  # 存储y坐标的四个角
    xc, yc, area, slices = clip_multi(pxs, pys, [2048, 2048])  # 剪切多边形以获取有效区域

    # 记录每个区域投影到WFSS像素网格中的波长
    tlams = np.zeros(len(xc))  # 初始化波长数组
    for i in range(len(slices)):
        tlams[slices[i]] = lams[i]  # 将波长分配给相应的区域

    # 存储光通量、波长以及它们在WFSS像素网格中的位置
    # 注意xcs和ycs中的值不是唯一的
    xcs.extend(xc.tolist())  # 添加x坐标
    ycs.extend(yc.tolist())  # 添加y坐标
    flams.extend((flam * area).tolist())  # 添加光通量
    alams.extend(tlams.tolist())  # 添加波长

    # 保存以便后续绘图。仅用于下面的绘图
    all_pxs.append(pxs)  # 保存所有x坐标
    all_pys.append(pys)  # 保存所有y坐标
    all_flams.append(flam)  # 保存所有光通量
    all_counts.append(flam * C.SENS["+1"](tlams) * dlam * 10000)  # 保存计数

在这一点上，我们有一组WFSS像素的列表（xcs, ycs），这些像素上落入的通量（flams，以$F_{\lambda}$单位表示）以及它们所包含的光的波长（alams）。

在我们的模拟中，我们不想投影通量单位，而是希望使用DN/s，因此我们将输入的flams值转换为DN/s（使用我们在Box Extraction笔记本中进行逆操作时所用的反向关系，将提取的DN/s转换为$F_{\lambda}$通量单位）。

In [ ]:
# 注意：下面的10000因子是因为dlam的单位是微米，而我们希望使用埃（Angstroms），
# 因为反向灵敏度是按埃定义的。

s = C.SENS["+1"](alams)  # 获取在波长alams下的灵敏度

counts = flams * s * dlam * 10000  # 计算计数值，考虑到灵敏度和波长间隔的转换

In [ ]:
# 打印出需要合并成最终WFSS像素网格的分散像素位数
print(f"There are {len(counts)} dispersed bits of pixels to combine into a final WFSS pixel grid")

我们现在有一份大量的 DN/s 值列表，以及这些值应该添加到我们的模拟 WFSS 观测中的位置，以便模拟我们源的完整分散光谱。

在 (xcs, ycs) 坐标列表中存在重复条目，因为不同波长被对象的“自我污染”混合在一起。

以下图表显示了分散的输入像素，使用蓝色轮廓，投影到最终的WFSS像素上，后者以灰色网格表示。分散的像素根据其通量（以DN/s为单位）以黑色阴影显示。

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 3))  # 创建一个15x3英寸的子图

for i in tqdm.tqdm(range(len(all_pxs))):  # 遍历所有像素数据

    for j in range(len(all_pxs[i])):  # 遍历每个像素的坐标

        tx = all_pxs[i][j][:]  # 获取当前像素的x坐标
        tx.append(tx[0])  # 将第一个x坐标添加到末尾，以闭合多边形

        ty = all_pys[i][j][:]  # 获取当前像素的y坐标
        ty.append(ty[0])  # 将第一个y坐标添加到末尾，以闭合多边形

        plt.plot(tx, ty, color='b', alpha=0.02)  # 绘制当前像素的边界，设置颜色和透明度

        c = all_counts[i]  # 获取当前像素的计数值
        c[c < 0] = 0  # 将负值计数设置为0，以避免填充错误

        plt.fill(tx, ty, color='k', alpha=c[j])  # 填充当前像素的区域，设置颜色和透明度

plt.grid()  # 显示网格

plt.xlim(200, 240)  # 设置x轴范围为200到240（可以改为(0, 700)查看整个光谱）

plt.xticks(range(200, 240))  # 设置x轴刻度

plt.xlabel("Dispersion coordinate (pixel)")  # 设置x轴标签

plt.ylabel("Cross dispersion coordinate (pixel)")  # 设置y轴标签

为了快速将这些计数在各自的 WFSS 像素位置上合并，我们可以使用 `scipy.coo_matrix`，这是一种快速且高效的方法：

In [ ]:
xcs = np.array(xcs)  # 将x坐标列表转换为NumPy数组

ycs = np.array(ycs)  # 将y坐标列表转换为NumPy数组

# 忽略超出探测器范围的计数和坐标

ok = (xcs >= 0) & (xcs < 2048) & (ycs >= 0) & (ycs < 2048)  # 创建布尔数组，标记有效的坐标

simulated = coo_matrix((counts[ok], (ycs[ok], xcs[ok])), shape=(2048, 2048)).toarray()  # 创建稀疏矩阵并转换为数组

显示该源的二维模拟光谱。我们可以看到在视场中有一条几乎水平的轨迹。将图表的x范围增加到(0, 700)将显示完整的轨迹。

In [ ]:
# 将x轴范围更改为(0, 700)以查看整个光谱
show_2d_spec(simulated, xlim=(200, 241), ylim=(1705, 1730), vmin=0, vmax=4)  # 显示二维光谱，设置x轴和y轴的范围，以及最小值和最大值

显示相同尺度下的真实数据。该轨迹也呈现为几乎水平的线。真实数据中的信号水平略高于模拟数据的信号水平。

In [ ]:
# 将x轴范围更改为(0, 700)以查看整个光谱
show_2d_spec(wfss_data, xlim=(200, 241), ylim=(1705, 1730), vmin=0, vmax=4)  # 显示二维光谱数据，设置x轴和y轴的范围，以及最小值和最大值

检查我们模拟图像的一个好方法是将模拟数据从我们的实际数据中减去。下图显示了模拟与真实差异图像。可以看到峰值水平的差异。模拟和真实轨迹的对齐也很明显，因为差异的宽度从左到右是恒定的。

In [ ]:
# 将x轴范围更改为(0, 700)以查看整个光谱
show_2d_spec(wfss_data - simulated, xlim=(200, 241), ylim=(1705, 1730), vmin=0, vmax=4)  # 显示2D光谱，设置x轴和y轴的范围，以及颜色映射的最小值和最大值

在色散方向上对模拟光谱和真实数据进行求和，以检查轨迹轮廓是否对齐。下图显示了求和后的真实数据（橙色）和模拟数据（蓝色线）。两个峰的左右对齐表明，模拟轨迹的位置与真实轨迹的位置非常接近。

In [ ]:
plt.plot(np.nansum(simulated, axis=-1), color='blue')  # 绘制模拟数据的总和，颜色为蓝色

plt.plot(np.nansum(wfss_data, axis=-1), color='orange')  # 绘制WFSS数据的总和，颜色为橙色

plt.xlim(1705, 1730)  # 设置x轴范围为1705到1730

plt.ylim(0, 3000)  # 设置y轴范围为0到3000

plt.grid()  # 显示网格

plt.ylabel("Summed Counts (DN/s)")  # 设置y轴标签

plt.xlabel("Cross dispersion coordinate (pixel)")  # 设置x轴标签

虽然模拟看起来在天文测量上是正确的，但我们没有考虑到分散背景，导致模拟中的信号过低。

这应该使用分散背景的模型来完成，但在这里，为了简化并且因为我们正在观察一个我们知道分散背景相对平坦且没有特征的区域，我们可以仅使用模拟来创建一个掩膜，然后计算每个像素的背景水平。

下面我们在模拟图像中对信号水平高于0.001 DN/s的像素进行掩膜。这会掩盖轨迹。剩下的像素（在下方以黑色显示）将用于计算背景水平。

In [ ]:
# 被掩蔽的像素，设置为NaN，在显示中呈现为白色

mask = simulated > 0.0001  # 创建一个掩蔽数组，标记出大于0.0001的像素

tmp = simulated * 1.  # 创建一个临时数组，复制模拟数据

tmp[mask] = np.nan  # 将掩蔽数组中的像素设置为NaN

# 将xlim更改为(0, 700)以查看整个光谱

show_2d_spec(tmp, xlim=(200, 241), ylim=(1705, 1730), vmin=0, vmax=1)  # 显示二维光谱图，设置x和y轴的范围以及颜色映射的最小值和最大值

计算中位数分散背景水平。

In [ ]:
bck_level = np.nanmedian(wfss_data[~mask])  # 计算去除掩膜后的wfss_data的中位数，作为背景水平

print(f"Background extimated to be {bck_level} DN/s per pixel")  # 打印估计的背景值，单位为DN/s每像素

现在绘制合并的模拟数据（蓝色）和背景扣除后的真实数据（橙色）。对齐良好的峰值更清晰地表明模拟数据和真实数据的轨迹对齐良好。模拟数据的峰值低于真实数据的峰值，显示出模拟在真实轨迹中的信号略有低估。

In [ ]:
plt.plot(np.nansum(wfss_data - bck_level, axis=-1), color='orange')  # 绘制去除背景后的WFSS数据总和，颜色为橙色

plt.xlim(1705, 1730)  # 设置x轴范围为1705到1730

plt.ylim(-100, 2500)  # 设置y轴范围为-100到2500

plt.plot(np.nansum(simulated, axis=-1), color='blue')  # 绘制模拟数据的总和，颜色为蓝色

plt.grid()  # 添加网格线

plt.ylabel("Summed Counts (DN/s)")  # 设置y轴标签为“总计数（DN/s）”

plt.xlabel("Cross dispersion coordinate (pixel)")  # 设置x轴标签为“横向色散坐标（像素）”

在减去背景后，模拟光谱与真实数据非常吻合。现在可以使用二维模拟光谱来减去重叠光谱的污染，以及在尝试估计背景时遮罩分散的光谱轨迹。

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="空间望远镜标志" width="200px"/>